## Важно:

Пожалуйста, поддерживайте ваш код в хорошем состоянии, пишите комментарии, убирайте бесполезные ячейки, пишите модели в специально отведенных модулях. Проверяющие могут **НА СВОЕ УСМОТРЕНИЕ** снижать баллы за:

1. Говнокод
2. Неэффективные решения
3. Вермишель из ячеек в тетрадке
4. Все остальное что им не понравилось


## Важно 2 (0 - 0.15 балла):

За использование логгеров типа wandb/comet/neptune и красивую сборку этой домашки в виде графиков/картинок в этих логгерах мы будем выдавать бонусные баллы.



## Важно 3:

Решением домашки является архив с использованными тетрадками/модулями, а так же **.pdf файл** с отчетом по проделанной работе по каждому пункту задачи. 
В нем необходимо описать какие эксперименты вы производили чтобы получить результат который вы получили, а так же обосновать почему вы решили использовать штуки которые вы использовали (например дополнительные лоссы для стабилизации, WGAN-GP, а не GAN/WGAN+clip)


In [1]:
%%bash
rm -rf model/
git clone https://github.com/isadrtdinov/StarGAN.git
mv StarGAN/* .
rm -r StarGAN/

Cloning into 'StarGAN'...


In [3]:
import numpy as np
import torchvision
import torchvision.transforms as T

import torch
from torch import nn
from torch.nn import functional as F

import os
import wandb
from tqdm import tqdm
from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from config import Params
import utils

params = Params()
utils.set_random_seed(params.random_seed)
wandb.init(project=params.project, config=params.__dict__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

В этом домашнем задании мы будем работать с Celeba. Celeba - это уже известный вам датасет состоящий из фотографий селеб в их привычной местности:

In [6]:
from dataset import CelebA

In [7]:
transform = T.Compose([
    T.RandomHorizontalFlip(0.5),
    T.CenterCrop(params.crop_size),
    T.Resize(params.img_size),
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [8]:
train_dataset = CelebA('celeba', attributes=params.attributes, target_type='attr',
                       split='train', transform=transform, download=False)
test_dataset = CelebA('celeba', attributes=params.attributes, target_type='attr',
                      split='test', transform=transform, download=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=params.batch_size,
                                           shuffle=True, drop_last=True, pin_memory=True,
                                           num_workers=params.num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=params.batch_size,
                                           shuffle=True, drop_last=True, pin_memory=True,
                                           num_workers=params.num_workers)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
from google.colab import drive
drive.mount('/content/drive/')
checkpoint_root = '/content/drive/MyDrive/checkpoints'

Mounted at /content/drive/


В этой домашней работе вам предлагается повторить результаты статьи StarGAN (https://arxiv.org/abs/1711.09020). 

Основная часть домашнего задания - чтение статьи и улучшение результатов, поэтому обязательно прочитайте не только StarGAN, но и другие Image-to-Image GAN подходы того времени (17-18 год) 


## Задача 1 (0.4 балла):

Повторить результаты StarGAN используя только CelebA

что это значит: в статье предлагается способ использовать несколько датасетов и выучивание аттрибутов уникальных для какого-то одного датасета. Мы не просим вас это делать, вам достаточно просто обучить StarGAN на CelebA

In [10]:
from model.stargan import StarGAN
from model.fid import FrechetInceptionDistance

model = StarGAN(params, device)
wandb.watch([model.net_G, model.net_D])
fid = FrechetInceptionDistance(device, batch_size=params.batch_size)

In [11]:
model.load_checkpoint(os.path.join(checkpoint_root, params.checkpoint_template.format(46)))

In [ ]:
orig_images, example_images, example_labels = \
    utils.generate_examples(test_dataset, example_ids=params.example_ids,
                            example_domains=params.example_domains)

In [ ]:
for epoch in range(41, params.num_epochs + 1):
    model.train()
    for real_images, src_labels in tqdm(train_loader, desc=f'Epoch {epoch}/{params.num_epochs}'):
        trg_labels = utils.permute_labels(src_labels)
        model.train_D(real_images, src_labels, trg_labels)
        model.train_G(real_images, src_labels, trg_labels)

        if model.train_step % params.log_steps == 0:
            wandb.log(model.metrics)
    
    if epoch % params.valid_epochs == 0:
        fid_score = fid.fid_score(model, test_loader)
        generated_images = model.generate(example_images, example_labels).cpu()
        example_grid = utils.process_examples(orig_images, generated_images,
                                              params.example_domains)

        wandb.log({
            'example': wandb.Image(example_grid),
            'FID score': fid_score,
            'train step': model.train_step,
            'epoch': epoch
        })
    
    if epoch % params.checkpoint_epochs == 0:
        checkpoint_file = params.checkpoint_template.format(epoch)
        model.save_checkpoint(os.path.join(checkpoint_root, checkpoint_file))

    clear_output()

## Важно 4: 

Если вы учите на колабе или на наших машинках, вероятнее всего что обучение будет очень долгим на картинках 256х256. Никто не мешает уменьшить разрешение, главное чтобы было видно что трансформации выучились

Еще, кажется что не все аттрибуты селебы являются очень важными или достаточно представленными в датасете. Не запрещается убирать бесполезные аттрибуты (только обоснуйте почему так сделали в отчете)

Не забывайте про аугментации

## Важно 5: 

Да, мы знаем что в на гитхабе лежить готовый код на путорче для этой статьи. Проблема в том что он написал на torch 0.4, поэтому, если мы увидим что вы используете __старый__ код со старыми модулями, то мы:

1. Будем неодобрительно смотреть
2. За наглое списывание будем снимать баллы


## Задача 2 (0.2 балла): 

Мерить качество на глаз - плохая идея. Подключите подсчет FID для каждой N эпохи, чтобы вы могли следить за прогрессом модели.

Сранение моделей между собой тоже возможно только по FID, поэтому трекайте его когда будете делать другие эксперименты

## Задача 3 (0.4 балла):

Если вы будете дословно повторять архитектуру авторов статьи, вы сразу же увидите что обучение станет дико долгим и не очень стабильным. Возможно у вас получится предложить несколько улучшений, которые приведут к хорошему FID, к визуально лучшим результатам или к более стабильному обучению.

В этой задаче хочется чтобы вы попробовали улучшить результаты статьи используя либо то что уже знаете, либо что-то из релевантных статей по Im2Im современности

## Важно 6: 

Когда вы будете показывать визуальные трансформации которые делает ваш StarGAN, хорошей идеей будет сразу же зафиксировать набор картинок (очевидно из валидации) и набор трансформаций на которых вы будете показывать результаты. Например: 10 картинок разных людей на которых вы покажете Male-Female, Beard-noBeard, Old-Young трансформации

## Важно 7 (0.15 балла): 

Выдам дополнительные баллы если у вас получится визуально красивая перекраска волос в разные цвета